In [37]:
import numpy as np

def sigmoid(x) -> np.ndarray:
    return 1 / (1 + np.exp(-x))


'''
# Todo List

- Add loss function
- Add bias
- Add Gradient Descent

'''
class np_lstm:
    def __init__(self):
        self.x = np.random.random((10, 5))
        self.W_ii = np.random.random((8, 5))
        self.W_if = np.random.random((8, 5))
        self.W_ig = np.random.random((8, 5))
        self.W_io = np.random.random((8, 5))
        
        self.W_hi = np.random.random((8, 8))
        self.W_hf = np.random.random((8, 8))
        self.W_hg = np.random.random((8, 8))
        self.W_ho = np.random.random((8, 8))
        self.c_0 = np.random.random((8,))
        self.h_0 = np.random.random((8,))
        
        self.total_hidden_states = []
    
    def run(self):
        for x in self.x:
            '''input'''
            i_t = sigmoid(np.matmul(x, self.W_ii.T)\
                          + np.matmul(self.h_0, self.W_hi.T))
            
            '''forget'''
            f_t = sigmoid(np.matmul(x, self.W_if.T)\
                          + np.matmul(self.h_0, self.W_hf.T))
            
            '''cell'''
            g_t = np.tanh(np.matmul(x, self.W_ig.T)\
                          + np.matmul(self.h_0, self.W_hg.T))
            
            '''output'''
            o_t = sigmoid(np.matmul(x, self.W_io.T)\
                          + np.matmul(self.h_0, self.W_ho.T))
            
            '''update cell and hidden'''
            self.c_0 = f_t * self.c_0 + i_t * g_t
            self.h_0 = o_t * np.tanh(self.c_0)
            
            self.total_hidden_states.append(self.h_0)
            
        return np.array(self.total_hidden_states), self.h_0, self.c_0

In [38]:
lstm = np_lstm()
outputs, h_t, c_t = lstm.run()
print(outputs.shape)
print(h_t.shape)
print(c_t.shape)

(10, 8)
(8,)
(8,)


In [1]:
import torch
import torch.nn as nn

'''

Inputs : input, (h_0, c_0) 
input of shape (seq_len, N, input_size)
if (h_0, c_0) is not provide : h_0 and c_0 default to zero
(h_0, c_0) of shape (num_layers * num_direction, N, hidden)

Outputs : output, (h_n, c_n)
output of shape (seq_len, N, num_direction*hidden)

if batch_first:
    intput and output tensors are provide as (N, seq, features)


'''

input_size = 5
hidden_size = 8

inputs = torch.Tensor(1, 10, input_size)#N, time, input
lstm = nn.LSTM(input_size, hidden_size, batch_first=True)#same as rnn

In [10]:
outputs, h_c = lstm(inputs)
h_t, c_t = h_c
print(outputs.size())
print(h_t.size())
print(c_t.size())

torch.Size([1, 10, 8])
torch.Size([1, 1, 8])
torch.Size([1, 1, 8])


In [11]:
'''
lstm has four times more gates(hidden) than rnn.

w_ih = Parameter(torch.Tensor(gate_size, layer_input_size))
w_hh = Parameter(torch.Tensor(gate_size, real_hidden_size))
b_ih = Parameter(torch.Tensor(gate_size))
b_hh = Parameter(torch.Tensor(gate_size))

길이가 32인건 
w_ih = (W_ii|W_if|W_ig|W_io) 각각 8개씩 4개라서 그럼.
w_hh = (W_hi|W_hf|W_hg|W_ho) 

ih = input - hidden
hh = hidden - hidden

개별 레이어 확인 방법
-> lstm.weight.ih_lk (kth layer)
'''

print(lstm._all_weights)

for n, w in zip(lstm._all_weights[0], lstm._flat_weights):
    print(f'size of {n} is {w.size()}') 

[['weight_ih_l0', 'weight_hh_l0', 'bias_ih_l0', 'bias_hh_l0']]
size of weight_ih_l0 is torch.Size([32, 5])
size of weight_hh_l0 is torch.Size([32, 8])
size of bias_ih_l0 is torch.Size([32])
size of bias_hh_l0 is torch.Size([32])
